# Process the WCST data

Transform to a suitable format for final processing.

----
```
author:     Zach Wolpe
email:      zachcolinwolpe@gmail.com
date:       21 January 2022
```
----

In [1]:
import sys
sys.path.append('../process data')
from dependencies import *
# import nbformatx

In [2]:
os.getcwd()

loc = '/Users/zachwolpe/Documents/Production/Dynocog/Python Implementation/final instance/model-free analysis/final_dataframes'
os.listdir(loc)

['wcst.pkl',
 'covariates.csv',
 'wcst_raw_data.pkl',
 'covariates.pkl',
 'wcst_raw_data.csv',
 'wcst.csv']

# encode choices

In [3]:
# encode choice cards --------*
choice_cards = pd.DataFrame({'card':[1,2,3,4], 'colour':['red','green','blue','yellow'], 'shape':['circle', 'triangle', 'cross', 'star'], 'number':[1,2,3,4]})
choice_cards

,card,colour,shape,number
0,1,red,circle,1
1,2,green,triangle,2
2,3,blue,cross,3
3,4,yellow,star,4


In [4]:
wcst_raw  = pd.read_pickle(loc + '/wcst_raw_data.pkl')
wcst_raw  = wcst_raw[['participant', 'rule', 'card_shape', 'card_number', 'card_colour', 'status','card_selected']]

# wcst_raw

In [6]:
matching_rule = []
for idx in tqdm(wcst_raw.index):

    sub = wcst_raw.loc[idx,]

    # extract info about card selected
    card_spec = choice_cards.loc[choice_cards.card==sub['card_selected'],]
    col = card_spec['colour'].values[0]
    shp = card_spec['shape'].values[0]
    nmb = card_spec['number'].values[0]

    if sub.status == 1: matching_rule.append(sub.rule)
    
    else:
        # find matching rule
        if sub.card_shape==shp or sub.card_number==nmb or sub.card_colour==col:
            if sub['card_shape']    == shp: matching_rule.append('shape')
            if sub['card_number']   == nmb: matching_rule.append('number')
            if sub['card_colour']   == col: matching_rule.append('color')

        else: matching_rule.append('na')
    # else: matching_rule.append('number') # placeholder: imputer



wcst_raw['matching_rule'] = matching_rule
wcst_raw


100%|██████████| 27400/27400 [00:06<00:00, 4247.62it/s]


,participant,rule,card_shape,card_number,card_colour,status,card_selected,matching_rule
0,816404.0,shape,triangle,1,yellow,1,1,shape
1,816404.0,shape,triangle,3,red,1,3,shape
2,816404.0,shape,cross,2,red,1,2,shape
3,816404.0,shape,triangle,1,yellow,1,1,shape
4,816404.0,shape,triangle,1,yellow,1,1,shape
...,...,...,...,...,...,...,...,...
27395,684712.0,color,triangle,4,blue,1,4,color
27396,684712.0,color,circle,4,green,1,4,color
27397,684712.0,color,circle,4,green,1,4,color
27398,684712.0,color,cross,2,red,1,2,color


In [7]:
# wcst_raw.loc[wcst_raw.participant==816404.0, ]
x = wcst_raw.copy()
x['correct'] = x.status == 1
x

,participant,rule,card_shape,card_number,card_colour,status,card_selected,matching_rule,correct
0,816404.0,shape,triangle,1,yellow,1,1,shape,True
1,816404.0,shape,triangle,3,red,1,3,shape,True
2,816404.0,shape,cross,2,red,1,2,shape,True
3,816404.0,shape,triangle,1,yellow,1,1,shape,True
4,816404.0,shape,triangle,1,yellow,1,1,shape,True
...,...,...,...,...,...,...,...,...,...
27395,684712.0,color,triangle,4,blue,1,4,color,True
27396,684712.0,color,circle,4,green,1,4,color,True
27397,684712.0,color,circle,4,green,1,4,color,True
27398,684712.0,color,cross,2,red,1,2,color,True


In [8]:
# Rt (correct action) --------* 
wcst_raw['reward'] = [int(i) for i in wcst_raw.status == 1]

# encode actions -------------*
encoding =  {'shape':1, 'number':2, 'color':3, 'na':0}
wcst_raw['action'] = None
for k in encoding.keys(): wcst_raw['action'][wcst_raw.matching_rule==k] = encoding[k]

# # check ----------------------*
wcst = wcst_raw[['participant', 'reward', 'status', 'action', 'rule', 'matching_rule']]
wcst['n_t'] = wcst_raw[['participant']].reset_index()['index'] % 100
wcst

,participant,reward,status,action,rule,matching_rule,n_t
0,816404.0,1,1,1,shape,shape,0
1,816404.0,1,1,1,shape,shape,1
2,816404.0,1,1,1,shape,shape,2
3,816404.0,1,1,1,shape,shape,3
4,816404.0,1,1,1,shape,shape,4
...,...,...,...,...,...,...,...
27395,684712.0,1,1,3,color,color,95
27396,684712.0,1,1,3,color,color,96
27397,684712.0,1,1,3,color,color,97
27398,684712.0,1,1,3,color,color,98


In [223]:

# save -------------------------------------*
loc = '/Users/zachwolpe/Documents/Production/Dynocog/Python Implementation/final instance/model-free analysis/final_dataframes/'
wcst.to_pickle(loc + 'wcst.pkl')
wcst.to_csv(loc + 'wcst.csv')

In [11]:
wcst.loc[wcst.status!=1, ]

,participant,reward,status,action,rule,matching_rule,n_t
10,816404.0,0,2,2,number,number,10
11,816404.0,0,2,2,number,number,11
12,816404.0,0,2,3,number,color,12
19,816404.0,0,2,1,color,shape,19
20,816404.0,0,2,3,color,color,20
...,...,...,...,...,...,...,...
27356,684712.0,0,2,1,shape,shape,56
27362,684712.0,0,2,1,color,shape,62
27370,684712.0,0,2,2,number,number,70
27371,684712.0,0,2,3,number,color,71
